In [38]:
inp = open('day12.txt', 'r').read()

In [41]:
sections = inp.split('\n\n')
shapes_text = sections[:-1]
regions_text = sections[-1].splitlines()

shapes = []
for shape_text in shapes_text:
    shape = []
    for line in shape_text.splitlines()[1:]:
        shape.append(line)
    
    orientations = []
    for _ in range(4):
        shape = [''.join(row) for row in zip(*shape[::-1])]
        if shape not in orientations:
            orientations.append(shape)
    shapes.append(orientations)

regions = []
for region_text in regions_text:
    region = []
    size, *quants = region_text.split()
    sx, sy = map(int, size[:-1].split('x'))
    quants = list(map(int, quants))
    regions.append((sx, sy, quants))


def can_place(shape, grid, x, y):
    for dy, row in enumerate(shape):
        for dx, cell in enumerate(row):
            if cell == '#':
                if y + dy >= len(grid) or x + dx >= len(grid[0]) or grid[y + dy][x + dx] != '.':
                    return False
    return True

def place(shape, grid, x, y, char):
    for dy, row in enumerate(shape):
        for dx, cell in enumerate(row):
            if cell == '#':
                grid[y + dy][x + dx] = char

def backtrack(shapes, shape_counts, grid, index):
    if index == len(shapes):
        return True
    
    shape = shapes[index]
    for orientation in shape:
        for y in range(len(grid)):
            for x in range(len(grid[0])):
                if can_place(orientation, grid, x, y):
                    place(orientation, grid, x, y, '#')
                    shape_counts[index] -= 1

                    if backtrack(shapes, shape_counts, grid, index + (shape_counts[index] == 0)):
                        return True
                    
                    place(orientation, grid, x, y, '.')
                    shape_counts[index] += 1
    return False

from tqdm import tqdm
def run(region):
    sx, sy, quants = region

    if sx*sy <= sum((sum(row.count('#') for row in shape[0]) * q for shape, q in zip(shapes, quants))):
        return 0
    if sx*sy >= sum(quants) * 9:
        return 1
    grid = [['.'] * sx for _ in range(sy)]
    index = min(i for i, q in enumerate(quants) if q > 0)
    return backtrack(shapes, quants, grid, index)

sum(run(region) for region in tqdm(regions))




  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:00<00:00, 103652.64it/s]


565